<a href="https://colab.research.google.com/github/thehydrowave/Adversarial-diffusion-distillation-pokemon/blob/main/ADD_part2_Naruto_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Adversarial Diffusion Distillation (ADD) - Projet Naruto

Ce notebook implémente l'entraînement d'un modèle student rapide à partir d'un modèle Stable Diffusion LoRA (teacher), selon le papier [Adversarial Diffusion Distillation (ADD)](https://arxiv.org/abs/2311.17042).

In [1]:
import numpy as np
print(np.__version__)

from diffusers import StableDiffusionPipeline
print("✅ Tout fonctionne avec diffusers.")

2.0.2
✅ Tout fonctionne avec diffusers.


In [ ]:
# 📦 Installation des dépendances
!pip install diffusers transformers accelerate datasets timm --quiet

In [ ]:
# 🔧 Imports & chargement du modèle teacher
import torch
from diffusers import StableDiffusionPipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher = StableDiffusionPipeline.from_pretrained(
    'runwayml/stable-diffusion-v1-5',
    torch_dtype=torch.float16
).to(device)

# Charger les poids LoRA Naruto
teacher.unet.load_attn_procs('./naruto_lora_weights')
teacher.eval()

In [ ]:
# 🧠 Initialiser le student
import copy
student = copy.deepcopy(teacher)
student.scheduler.set_timesteps(num_inference_steps=4)
student.train()

In [ ]:
# 🤖 Discriminateur basé sur ViT (via timm)
import timm
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.head = nn.Linear(self.backbone.num_features, 1)

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

discriminator = Discriminator().to(device)

## 🔁 Boucle d'entraînement ADD simplifiée (à compléter selon dataset Naruto)

In [ ]:
# 🔄 Training loop (SQUELETTE À COMPLÉTER)
lambda_adv = 0.5
optimizer_student = torch.optim.Adam(student.unet.parameters(), lr=1e-5)
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=1e-5)

# ➕ À compléter : charger le dataset Naruto BLIP captions, générer x_s, calculer pertes
# ➕ Ajout de SDS loss + hinge loss sur output du discriminateur
# ➕ Entraînement alterné student/discriminator
